In [1]:
from scraping import *
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
# search each link 
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import math
import re
import time


dfolder='./downloads'
geko_path="../geckodriver"
link='https://www.booking.com/index.es.html'


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)


In [2]:
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath','//div[@class="d8f77e681c"]')
    #number of total hotel listings divided of listings p. page
    total_listings = int(re.findall(r'\d+', a[-1].text)[0])
    total_listings = min(total_listings, 1000)
    page_num = math.ceil(total_listings / 25)
    print(f'Total hotels: {total_listings} \nNumber of pages: {page_num}')
    return(page_num) 

In [26]:
def search_hotels(start_date, end_date, place, browser):
    browser.get('https://www.booking.com/index.es.html')
    xpath = '//*[@id="onetrust-reject-all-handler"]'

    time.sleep(10)
    #accept cookies
    check_and_click(browser, xpath, type='xpath')
    time.sleep(5)

    # refresh page to trigger pop up then exit pop up (if it appears)
    browser.refresh()
    browser.refresh()
    browser.refresh()
    time.sleep(10)
    check_and_click(browser, '//button[@aria-label="Ignorar información sobre el inicio de sesión."]', type='xpath')

    # Switch to the iframe and exit the google long in pop up
    try:
        iframe_path  = (By.XPATH, '//iframe[@title="Cuadro de diálogo Iniciar sesión con Google"]')
        iframe = WebDriverWait(browser, 10).until(EC.frame_to_be_available_and_switch_to_it(iframe_path))

        element_locator = (By.XPATH, '//div[@aria-label="Cerrar"]')
        element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable(element_locator))
        element.click()

        browser.switch_to.default_content()
    except Exception as e:
        print(str(e))

        
    # enter location
    search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
    search1.send_keys(place)

    #click date selector
    css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'
    browser.find_element('css selector',css).click()


    #click start and end date
    start_date_found = False
    end_date_found = False
    while start_date_found == False or end_date_found == False:
        path='//span[@class="cf06f772fa"]'
        dates = browser.find_elements('xpath',path)
        for date in dates:
            if date.get_attribute("data-date") == start_date:
                date.click()
                start_date_found = True
            if date.get_attribute("data-date") == end_date:
                date.click()
                end_date_found = True
                break
        next_page_button = '//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 f671049264 deab83296e f4552b6561 dc72a8413c f073249358"]'
        check_and_click(browser, next_page_button, type='xpath')

    time.sleep(3)

    #click search button
    my_xpath='//span[text()="Buscar"]'
    check_obscures(browser,my_xpath , type='xpath')
    check_and_click(browser,my_xpath , type='xpath')

    time.sleep(10)
    check_and_click(browser, '//button[@aria-label="Ignorar información sobre el inicio de sesión."]', type='xpath')
    
    # filter only hotels and hostels
    hotels = '//input[@value="ht_id=204"]'
    # click first occurrence of hotels filter
    print('clicking hotels filter')
    check_and_click(browser, hotels, type='xpath')
    time.sleep(10)

    hostels = '//input[@value="ht_id=216"]'
    # click first occurrence of hostels filter
    print('clicking hostels filter')
    check_and_click(browser, hostels, type='xpath')
    time.sleep(10)
    
    # get all pages in search results
    pages = get_number_pages(browser)

    hotel_card_container = '//div[@data-testid="property-card"]'  # XPath expression to locate the hotel card container
    data = []  # List to store the extracted data for each hotel

    for page in range(pages):
        hotels = browser.find_elements('xpath', hotel_card_container)  # Find all hotel elements on each iteration
        for hotel in hotels:  # Iterate over each hotel element
            data_dic = {}  # Dictionary to store the extracted data for the current hotel

            try:
                # Extract the photo URL of the hotel
                photo_path = './/img[@data-testid="image"]'  # XPath expression to locate the photo element
                photo_url = hotel.find_element('xpath', photo_path).get_attribute('src')  # Get the src attribute of the photo element
                data_dic['photo_url'] = photo_url  # Store the photo URL in the dictionary
            except NoSuchElementException:
                print('No photo URL')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the link of the hotel
                link_path = './/a[@class="a78ca197d0"]'  # XPath expression to locate the link element
                link = hotel.find_element('xpath', link_path).get_attribute('href')  # Get the href attribute of the link element
                data_dic['link'] = link  # Store the link in the dictionary
            except NoSuchElementException:
                print('No link')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the name of the hotel
                name_path = './/div[@data-testid="title"]'  # XPath expression to locate the name element
                name = hotel.find_element('xpath', name_path).text  # Get the text content of the name element
                data_dic['name'] = name  # Store the name in the dictionary
            except NoSuchElementException:
                print('No name')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the central distance of the hotel
                central_distance_path = './/span[@data-testid="distance"]'  # XPath expression to locate the central distance element
                central_distance = hotel.find_element('xpath', central_distance_path).text  # Get the text content of the central distance element
                data_dic['central_distance'] = central_distance  # Store the central distance in the dictionary
            except NoSuchElementException:
                print('No central distance')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the sustainability information of the hotel
                sustainability_path = './/span[@class="abf093bdfe d068504c75 f68ecd98ea"]'  # XPath expression to locate the sustainability element
                sustainability = hotel.find_element('xpath', sustainability_path).text  # Get the text content of the sustainability element
                data_dic['sustainability'] = sustainability  # Store the sustainability information in the dictionary
            except NoSuchElementException:
                data_dic['sustainability'] = None
                print('No sustainability information')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the rating of the hotel
                rating_path = './/div[@class="a3b8729ab1 d86cee9b25"]'  # XPath expression to locate the rating element
                rating = hotel.find_element('xpath', rating_path).text  # Get the text content of the rating element
                data_dic['rating'] = rating  # Store the rating in the dictionary
            except NoSuchElementException:
                print('No rating')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the comfort rating of the hotel
                comfort_rating_path = './/span[@class="a3332d346a"]'  # XPath expression to locate the comfort rating element
                comfort_rating = hotel.find_element('xpath', comfort_rating_path).text  # Get the text content of the comfort rating element
                data_dic['comfort_rating'] = comfort_rating  # Store the comfort rating in the dictionary
            except NoSuchElementException:
                print('No comfort rating')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the review count of the hotel
                review_count_path = './/div[@class="abf093bdfe f45d8e4c32 d935416c47"]'  # XPath expression to locate the review count element
                review_count = hotel.find_element('xpath', review_count_path).text  # Get the text content of the review count element
                data_dic['review_count'] = review_count  # Store the review count in the dictionary
            except NoSuchElementException:
                print('No review count')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the room description of the hotel
                room_description_path = './/h4[@class="abf093bdfe e8f7c070a7"]'  # XPath expression to locate the room description element
                room_description = hotel.find_element('xpath', room_description_path).text  # Get the text content of the room description element
                data_dic['room_description'] = room_description  # Store the room description in the dictionary
            except NoSuchElementException:
                print('No room description')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the room specifics of the hotel
                room_specifics_path = './/div[@class="abf093bdfe"]'  # XPath expression to locate the room specifics elements
                room_specifics = hotel.find_elements('xpath', room_specifics_path)  # Find all room specifics elements
                room_specifics_text = []
                for room_specific in room_specifics:
                    room_specifics_text.append(room_specific.text)  # Get the text content of each room specifics element
                data_dic['room_specifics'] = ' '.join(room_specifics_text)  # Store the room specifics as a single string in the dictionary
            except NoSuchElementException:
                print('No room specifics')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the price of the hotel
                price_path = './/span[@data-testid="price-and-discounted-price"]'  # XPath expression to locate the price element
                price = hotel.find_element('xpath', price_path).text  # Get the text content of the price element
                data_dic['price'] = price  # Store the price in the dictionary
            except NoSuchElementException:
                print('No price')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the original price of the hotel (if available)
                original_price_path = './/span[@class="c73ff05531 e84eb96b1f"]'  # XPath expression to locate the original price element
                original_price = hotel.find_element('xpath', original_price_path).text  # Get the text content of the original price element
                data_dic['original_price'] = original_price  # Store the original price in the dictionary
            except NoSuchElementException:
                data_dic['original_price'] = None  # If the original price element is not found, store None in the dictionary
                print('No original price')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the neighborhood and city of the hotel
                neighborhood_city_path = './/span[@data-testid="address"]'  # XPath expression to locate the neighborhood and city element
                neighborhood_city = hotel.find_element('xpath', neighborhood_city_path).text  # Get the text content of the neighborhood and city element
                data_dic['neighborhood_city'] = neighborhood_city  # Store the neighborhood and city in the dictionary
            except NoSuchElementException:
                print('No neighborhood and city')
            except Exception as e:
                print(e)
                print('Something went wrong')

            data.append(data_dic)  # Add the dictionary to the data list
            data_dic = {}

        # double check page number to make sure it is not the last page
        max_page = get_number_pages(browser)
        if page > max_page:
            page = max_page
            
        # The data list now contains the extracted data for all hotels
        next_page_path = '//button[@aria-label="Página siguiente"]'
        next_page = browser.find_elements('xpath', next_page_path)
        next_page[0].click()
        time.sleep(5)
        check_and_click(browser, '//button[@aria-label="Ignorar información sobre el inicio de sesión."]', type='xpath')

    data = pd.DataFrame(data)
    data['start_date'] = start_date
    data['end_date'] = end_date
    return data
        

In [18]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_hotel_data(link):
    try:
        hotel_data = {}
        r = requests.get(link)
        soup = BeautifulSoup(r.content, 'html.parser')

        map_element = soup.find('a', id='hotel_sidebar_static_map').get('data-atlas-latlng')
        hotel_data['map_lat'] = map_element.split(',')[0]
        hotel_data['map_lng'] = map_element.split(',')[1]
        hotel_data['name'] = soup.find('h2', class_='d2fee87262 pp-header__title').get_text()
        hotel_data['description'] = soup.find('div', class_='hp-description').get_text()

        room_ratings = soup.find_all('div', class_='ccb65902b2')
        room_ratings_desc = [element.get_text() for element in room_ratings if ',' not in element.get_text()]
        room_ratings = [element.get_text() for element in room_ratings if ',' in element.get_text()]


        for index in range(0,len(room_ratings)): 
            hotel_data[f'room_rating_{room_ratings_desc[index]}'] = room_ratings[index]

        return hotel_data

    except Exception as e:
        print(f'Error processing {link}: {e}')
        return None

def get_hotel_details(data, max_threads=10):
    hotels_data = []

    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        future_to_url = {executor.submit(fetch_hotel_data, link): link for link in data.link}
        for future in as_completed(future_to_url):
            hotel_data = future.result()
            if hotel_data:
                hotels_data.append(hotel_data)
                print(f'Processed {len(hotels_data)} out of {len(data.link)}')

    hotels_data_df = pd.DataFrame(hotels_data)

    # join the data on the hotel name
    return data.join(hotels_data_df.set_index('name'), on='name')



In [24]:
browser.get('https://www.booking.com/index.es.html')
xpath = '//*[@id="onetrust-reject-all-handler"]'

time.sleep(10)

# enter location
search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
search1.send_keys(place)

#click date selector
css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'
browser.find_element('css selector',css).click()


#click start and end date
start_date_found = False
end_date_found = False
while start_date_found == False or end_date_found == False:
    path='//span[@class="cf06f772fa"]'
    dates = browser.find_elements('xpath',path)
    for date in dates:
        if date.get_attribute("data-date") == start_date:
            date.click()
            start_date_found = True
        if date.get_attribute("data-date") == end_date:
            date.click()
            end_date_found = True
            break
    next_page_button = '//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 f671049264 deab83296e f4552b6561 dc72a8413c f073249358"]'
    check_and_click(browser, next_page_button, type='xpath')

time.sleep(3)

#click search button
my_xpath='//span[text()="Buscar"]'
check_obscures(browser,my_xpath , type='xpath')
check_and_click(browser,my_xpath , type='xpath')

time.sleep(10)
check_and_click(browser, '//button[@aria-label="Ignorar información sobre el inicio de sesión."]', type='xpath')

# filter only hotels and hostels
hotels = '//input[@value="ht_id=204"]'
# click first occurrence of hotels filter
print('clicking hotels filter')
check_and_click(browser, hotels, type='xpath')
time.sleep(10)

hostels = '//input[@value="ht_id=216"]'
# click first occurrence of hostels filter
print('clicking hostels filter')
check_and_click(browser, hostels, type='xpath')
time.sleep(10)

# get all pages in search results
pages = get_number_pages(browser)

clicking hotels filter
clicking hostels filter
Total hotels: 482 
Number of pages: 20


In [25]:
pages

20

## Event week search in primary city

In [5]:
start_date = '2024-05-25'
end_date = '2024-06-02'
place = 'Barcelona'
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_evnt_bcn = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_evnt_bcn = get_hotel_details(data_evnt_bcn)

data_evnt_bcn = data_evnt_bcn.drop_duplicates()

data_evnt_bcn.to_csv(file_path)


Total hotels: 391 
Number of pages: 16
No original price
No original price
No original price
No original price
No original price
No original price
No original price
No sustainability information
No original price
No original price
No original price
No original price
No sustainability information
No comfort rating
No original price
No original price
No original price
No original price
No sustainability information
No original price
No comfort rating
No original price
No original price
No original price
No original price
No sustainability information
No original price
Total hotels: 391 
Number of pages: 16
No sustainability information
No original price
No comfort rating
No sustainability information
No original price
No original price
No original price
No original price
No original price
No sustainability information
No original price
No sustainability information
No original price
No original price
No original price
No original price
No sustainability information
No original price
No s

## Non event week search in primary city

In [27]:
start_date = '2024-05-11'
end_date = '2024-05-19'
place = 'Barcelona'
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_nonevnt_bcn = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_nonevnt_bcn = get_hotel_details(data_nonevnt_bcn)

data_nonevnt_bcn = data_nonevnt_bcn.drop_duplicates()
data_nonevnt_bcn.to_csv(file_path)

Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

clicking hotels filter
clicking hostels filter
Total hotels: 482 
Number of pages: 20
No original price
No sustainability information
No original price
No original price
No sustainability information
No comfort rating
No original price
No sustainability information
No original price
No original price
No sustainability information
No original price
No original price
No original price
No original price
No original price
No original price
No original price
No sustainability information
No comfort rating
No original price
No original price
No original price
No original price
No original price
No original price
No original price
No original price
No sustainability information
No rating

## Event week search in non primary city

In [29]:
start_date = '2024-05-25'
end_date = '2024-06-02'
place = 'Valencia'
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_evnt_vlc = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_evnt_vlc = get_hotel_details(data_evnt_vlc)

data_evnt_vlc = data_evnt_vlc.drop_duplicates()
data_evnt_vlc.to_csv(file_path)

Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

clicking hotels filter
clicking hostels filter
Total hotels: 131 
Number of pages: 6
No original price
No original price
No sustainability information
No comfort rating
No original price
No original price
No original price
No original price
No original price
No sustainability information
No original price
No sustainability information
No original price
No original price
No original price
No original price
No sustainability information
No comfort rating
No original price
No original price
No original price
No original price
No sustainability information
No original price
No comfort rating
No original price
No original price
No sustainability information
No original price
Total hote

## Non event week search in non primary city

In [30]:
start_date = '2024-05-11'
end_date = '2024-05-19'
place = 'Valencia'
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_nonevnt_vlc = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_nonevnt_vlc = get_hotel_details(data_nonevnt_vlc)

data_nonevnt_vlc = data_nonevnt_vlc.drop_duplicates()
data_nonevnt_vlc.to_csv(file_path)

Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

clicking hotels filter
clicking hostels filter
Total hotels: 166 
Number of pages: 7
No original price
No original price
No original price
No sustainability information
No comfort rating
No original price
No original price
No original price
No sustainability information
No comfort rating
No original price
No original price
No original price
No original price
No original price
No original price
No original price
No sustainability information
No original price
No sustainability information
No comfort rating
No original price
No original price
No original price
No original price
No sustainability information
No original price
Total hotels: 166 
Number of pages: 7
No original price
No